In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV

from kaggle_prediction_interval_birthweight.model.sampling_utils import (
    compute_highest_density_interval,
)
from kaggle_prediction_interval_birthweight.workflow.metric import score
from kaggle_prediction_interval_birthweight.data.data_processing import DataProcessor
from kaggle_prediction_interval_birthweight.model.linear_regression import (
    RidgeRegressor,
)
from kaggle_prediction_interval_birthweight.model.hist_gradient_boosting import (
    HistBoostRegressor,
)
from kaggle_prediction_interval_birthweight.model.neural_network import (
    MissingnessNeuralNet,
)
from kaggle_prediction_interval_birthweight.model.ensembler import (
    Ensembler,
)

In [3]:
data = pd.read_csv("../data/train.csv")
data_test = pd.read_csv("../data/test.csv")

In [ ]:
ensembler = Ensembler()
ensembler.fit(data)

lower_ensemble, upper_ensemble = ensembler.predict_intervals(data)

fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.vlines(x=data["DBWT"], ymin=lower_ensemble, ymax=upper_ensemble, alpha=0.25)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, color="orange")
plt.show()

Ensembler fold 1 of 3 begins.
Training the ridge regression model.
Training the histogram boosting model.
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Training the neural network model.


In [ ]:
train_processor = DataProcessor(model_type="linear regression")
X, y = train_processor(data)

test_processor = DataProcessor(
    model_type="linear regression",
    feature_categories=train_processor.feature_categories,
    standardization_parameters=train_processor.standardization_parameters,
)
X_test = test_processor(data_test)

ridge_regressor = RidgeRegressor().fit(X, y)


lower_ridge, upper_ridge = ridge_regressor.predict_intervals(X)

fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.vlines(x=data["DBWT"], ymin=lower_ridge, ymax=upper_ridge, alpha=0.25)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, color="orange")
plt.show()

In [ ]:
train_processor = DataProcessor(model_type="neural network")
X, y = train_processor(data)

nn = MissingnessNeuralNet(units=10, n_layers=1, verbose=1, n_epochs=200)
nn.fit(X, y)

lower_nn, upper_nn = nn.predict_intervals(X)

fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.vlines(x=data["DBWT"], ymin=lower_nn, ymax=upper_nn, alpha=0.25)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, color="orange")
plt.show()

In [ ]:
train_processor = DataProcessor(model_type="xgboost")
X, y = train_processor(data)

histboost_regressor = HistBoostRegressor()
histboost_regressor.fit(X, y)

lower_histboost, upper_histboost = histboost_regressor.predict_intervals(X)

fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.vlines(x=data["DBWT"], ymin=lower_histboost, ymax=upper_histboost, alpha=0.25)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, color="orange")
plt.show()

In [ ]:
lower, upper = compute_highest_density_interval(data["DBWT"])

print(f"Coverage is {data['DBWT'].between(lower, upper).mean() * 100:.1f}%")

data["DBWT"].hist(bins=100)
plt.axvline(lower, color="orange")
plt.axvline(upper, color="orange")
plt.show()

In [ ]:
nois, coverage = score(data["DBWT"].values, lower, upper, 0.9)

print(f"NOIS is {nois:.1f}, and coverage is {coverage*100:.1f}%")

In [ ]:
nois, coverage = score(data["DBWT"].values, lower_ridge, upper_ridge, 0.9)

print(f"NOIS is {nois:.1f}, and coverage is {coverage*100:.1f}%")

In [ ]:
nois, coverage = score(data["DBWT"].values, lower_histboost, upper_histboost, 0.9)

print(f"NOIS is {nois:.1f}, and coverage is {coverage*100:.1f}%")

In [ ]:
nois, coverage = score(data["DBWT"].values, lower_nn, upper_nn, 0.9)

print(f"NOIS is {nois:.1f}, and coverage is {coverage*100:.1f}%")

In [ ]:
nois, coverage = score(data["DBWT"].values, lower_ensemble, upper_ensemble, 0.9)

print(f"NOIS is {nois:.1f}, and coverage is {coverage*100:.1f}%")